<a href="https://colab.research.google.com/github/abbyrana/warzoneSouper/blob/main/WarzoneSoupMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<br>

# What I did?

*   Extracted data of 1500+ weapon attachments in Call of Duty Warzone from [GAme8](https://game8.co/games/Modern-Warfare-3/archives/431902) via web scraping using [BeautifulSoup4](https://beautiful-soup-4.readthedocs.io/en/latest/) python library.
*   Extracted list of all compatible weapons for all 1500+ attachments in a very small amount of time by applying the concept of [Multithreading](https://docs.python.org/3/library/threading.html).
*   Built an app on Google [AppSheet](https://about.appsheet.com/home/) which  manages attachment inventory and ranks the best guns which unlock the most best attachments.
*   Created database on Google Sheets which is used by the app.

<br>

<br>Prequisities

In [ ]:
!pip install beautifulsoup4
!pip install lxml

<br>Code

In [ ]:
from bs4 import BeautifulSoup
import concurrent.futures
import requests



# LevelUp class (weapon, level)
class LevelUp:
  def __init__(self, weapon, level):
    self.weapon=weapon
    self.level=level
  def show(self):
    print(f'{self.weapon} \t {self.level}')



# Attachment class (aname, atype, aurl, unlockby, equipin)
class Attachment:
  equip2 = []
  def __init__(self, aname, atype, aurl, unlockby):
    self.aname = aname
    self.atype = atype
    self.aurl = aurl
    self.unlockby = unlockby
  def equip(self, equip2):
    self.equip2 = equip2

  # Display funtions to copy data
  def showEquip2(self):
    wstr = f'{self.equip2[0]}'
    for i in range(1, len(self.equip2)):
      wstr+=f', {self.equip2[i]}'
    print(wstr)

  # for Unlock sheet
  def taname(self):
    for u in self.unlockby:
      print(self.aname)
  def taweapon(self):
    for u in self.unlockby:
      print(u.weapon)
  def talevel(self):
    for u in self.unlockby:
      print(u.level)



# Worker function for multithreading
def fetchEquip2(aobj):
  # to handle empty url
  if aobj.aurl is None:
    return ['']

  # fetch the table header for compatible guns
  reqa = requests.get(aobj.aurl)
  soupa = BeautifulSoup(reqa.content, 'lxml')
  ath = soupa.find('th', string=f'Guns That Can Use the Attachment')

  # to handle nonexisting data
  if ath is None:
    return ['']

  # get all table cells
  atbody = ath.parent.parent
  atds = atbody.select('tr > td')

  # extract the guns and put into a list
  equip2 = []
  for atd in atds:
    equip2.append(atd.get_text().strip())
  return equip2



# Get the attachment list page
urlAttachList = 'https://game8.co/games/Modern-Warfare-3/archives/431902'
reqAttachList = requests.get(urlAttachList)
soupAttachList = BeautifulSoup(reqAttachList.content, 'lxml')
arows = soupAttachList.select('body > div.l-content > div.l-3col > div.l-3colMain > div.l-3colMain__center.l-3colMain__center--shadow > div.archive-style-wrapper > table.a-table.a-table.a-table.flexible-cell > tbody > tr')



# Set all attachment's names, types, url and unlockby
attachments = []

for arow in arows:
  # Get attachment names types and url
  aname = arow.select_one('td:nth-child(1)').get_text().strip()
  atype = arow.select_one('td:nth-child(2)').get_text().strip()
  aurl = arow.select_one('td:nth-child(1) > a')
  if aurl is not None:
    aurl = aurl.get('href')

  # Get weapon and level list
  unlockby = []
  weps = arow.select('td:nth-child(3) > a')
  lvls = arow.select('td:nth-child(3) > b')
  if(len(weps) > 0):
    for i in range(len(weps)):
      wep = weps[i].get_text().strip()
      lvl = int(lvls[i].get_text().strip()[6:])
      unlockby.append(LevelUp(wep, lvl))

  # Set everything to attachment list
  attachments.append(Attachment(aname, atype, aurl, unlockby))

In [ ]:
# Multithreading Part takes about 5-7 minutes
with concurrent.futures.ThreadPoolExecutor() as executor:

  # Get all attachment's equip2
  wlists = executor.map(fetchEquip2, attachments)

  # Set all attachment's equip2
  i=0
  for wlist in wlists:
    attachments[i].equip(wlist)
    i+=1

<br>Actions

In [ ]:
# Show equip2 for all attachments
for a in attachments:
  a.showEquip2()

In [ ]:
# Show aurl for all attachments
for a in attachments:
  print(a.aurl)

In [ ]:
# Show aname for Attachment column in GoogleSheets
for a in attachments:
  a.taname()

In [ ]:
# Show weapon for Weapon column in GoogleSheets
for a in attachments:
  a.taweapon()

In [ ]:
# Show level for Level column in GoogleSheets
for a in attachments:
  a.talevel()

<br>

# Challenges
*   Getting a specific html tag using one css selector accross diffrent pages due to inconsistent ordering of tag elements.<br>
FIX: Use find() method to search a specific string to get the desired tag.

* Dealinjg with None type errors due to absence of desired data on the webpage.
FIX: Use if statement to skip the None type values.

*   Trying not to make too many requests.<br>
FIX: Write and test code in a block that is separeted from the request calls on Google .

*   Getting a specific column value of a key from the a diffrent table in Google Sheets.<br>
FIX: Use lookup() or vlookup() function.

*   Coming up with a ranking system that ranks and groups weapons from highest amount of attachment unlock to least and from lowest level to highest level requirement.<br>
FIX: Make a column that appends sum of all attachment's points that the weapon unlocks, weapon name and 100 minus the level requirment. Sort the table by this column in decending  order.

<br>